In [ ]:
# Note: ChatGPT assisted in generating the following cell code for visualization.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorboard.backend.event_processing import event_accumulator

# --- Helper Function to Load TensorBoard Data ---
def load_tensorboard_df(log_dir, tag='rollout/ep_rew_mean'):
    ea = event_accumulator.EventAccumulator(log_dir, size_guidance={'scalars': 0})
    ea.Reload()
    events = ea.Scalars(tag)
    if not events:
        print(f"No events found for tag '{tag}' in {log_dir}")
    df = pd.DataFrame(
        [(e.wall_time, e.step, e.value) for e in events],
        columns=['wall_time', 'step', 'value']
    )
    return df

# --- Define Base Directories ---
base_log_dir = '/mnt/c/Users/SHiFT/OneDrive/Documents/dev/Kane vs Abel Mortal Kombat II/experiments/logs/DuelingDDQN'
exp_lr_base = os.path.join(base_log_dir, 'DuelingDoubleDQN_4M_VeryEasyVsJax_ExpLr')
ln_lr_base  = os.path.join(base_log_dir, 'DuelingDoubleDQN_4M_VeryEasyVsJax_LnLr')

# --- Create List of Log Directories for Each Condition ---
exp_lr_dirs = [os.path.join(exp_lr_base, d) for d in os.listdir(exp_lr_base) 
               if os.path.isdir(os.path.join(exp_lr_base, d))]
ln_lr_dirs  = [os.path.join(ln_lr_base, d) for d in os.listdir(ln_lr_base) 
               if os.path.isdir(os.path.join(ln_lr_base, d))]

print("Exponential LR directories:", exp_lr_dirs)
print("Linear LR directories:", ln_lr_dirs)

def aggregate_runs(log_dirs, tag='rollout/ep_rew_mean'):
    dfs = []
    for log_dir in log_dirs:
        df = load_tensorboard_df(log_dir, tag=tag)
        print(f"Loaded {len(df)} events from {log_dir}")
        if not df.empty:
            dfs.append(df)
    if not dfs:
        raise ValueError("No valid data found in any of the provided log directories.")
    merged_df = pd.DataFrame({'step': dfs[0]['step']})
    for i, df in enumerate(dfs):
        merged_df[f'value_run{i}'] = df['value'].values
    value_columns = [col for col in merged_df.columns if col.startswith('value_run')]
    merged_df['mean'] = merged_df[value_columns].mean(axis=1)
    merged_df['std'] = merged_df[value_columns].std(axis=1, ddof=1)
    return merged_df

# --- Aggregate Data for Both Conditions ---
df_exp_agg = aggregate_runs(exp_lr_dirs, tag='rollout/ep_rew_mean')
df_ln_agg  = aggregate_runs(ln_lr_dirs, tag='rollout/ep_rew_mean')

# --- Set Professional Plot Style ---
sns.set_context("paper", font_scale=1.2)
sns.set_style("whitegrid")
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 1.0,
    'lines.markersize': 4,
    'legend.fontsize': 10,
    'figure.dpi': 300,
})

# --- Plot Only the Standard Deviation (Std) Curves ---
plt.figure(figsize=(12, 6), dpi=300)

# Color choices (colorblind-friendly)
exp_color = '#0072B2'
ln_color  = '#D55E00'

plt.plot(
    df_exp_agg['step'], df_exp_agg['std'],
    label='Exponential LR Std',
    color=exp_color,
    linestyle='--'
)
plt.plot(
    df_ln_agg['step'], df_ln_agg['std'],
    label='Linear LR Std',
    color=ln_color,
    linestyle='--'
)

plt.xlabel('Training Steps')
plt.ylabel('Standard Deviation of Episode Reward Mean')
plt.title('Standard Deviation Comparison: Exponential vs. Linear LR\n(Liu Kang vs. Jax, Very Easy CPU Opponent)')
plt.legend(loc='upper left', frameon=True)
plt.grid(True)
plt.tight_layout()
plt.savefig('std_comparison.pdf', format='pdf', bbox_inches='tight')
plt.show()


Exponential LR directories: []
Linear LR directories: []


ValueError: No valid data found in any of the provided log directories.